# Captcha Solver


## Samples

- https://keras.io/examples/vision/captcha_ocr

- https://virgool.io/dataio/how-to-break-a-golestan-captcha-system-with-machine-learning-kukzjnwwsqdx

- https://github.com/AmirH-Moosavi/Golestan

- https://pyimagesearch.com/2021/07/14/breaking-captchas-with-deep-learning-keras-and-tensorflow/

- https://medium.com/@ageitgey/how-to-break-a-captcha-system-in-15-minutes-with-machine-learning-dbebb035a710

## OpenCV

main site: https://opencv.org

docs site: https://docs.opencv.org

Note: C++ and Python docs are mixed.

OpenCV 4.10.0 tutorial for Python:
https://docs.opencv.org/4.10.0/d6/d00/tutorial_py_root.html


### remove noise and lines:

[Image Thresholding](https://docs.opencv.org/4.10.0/d7/d4d/tutorial_py_thresholding.html)

[Morphological Transformations](https://docs.opencv.org/4.10.0/d9/d61/tutorial_py_morphological_ops.html)

[Smoothing Images](https://docs.opencv.org/4.10.0/d4/d13/tutorial_py_filtering.html)

https://stackoverflow.com/questions/71425968/remove-horizontal-lines-with-open-cv


image binarization


## Basic concepts


- Gradient descent
- Batch Gradient Descent
- Mini Batch Gradient Descent
- Stochastic gradient descent

### loss function

### backpropagation

### Optimizers

https://keras.io/api/optimizers/

- SGD
- Adagrad
- Adadelta
- RMSprop
- Adam
- Nadam

## Keras

### Model training 

https://keras.io/api/models/model_training_apis/

### Callbacks 

https://keras.io/api/callbacks/

- `ModelCheckpoint`
- `EarlyStopping`
- `ReduceLROnPlateau`

### Metrics

https://keras.io/api/metrics/

### Saving Model
Two ways to save and load keras model:

- [ModelCheckpoint](https://keras.io/api/callbacks/model_checkpoint): Callback to save the Keras model or model weights at some frequency.


- [Whole model saving & loading](https://keras.io/api/models/model_saving_apis/model_saving_and_loading): Saves a model as a .keras file.

  ```python
  import keras

  model = keras.Sequential(
      [
          keras.layers.Dense(5, input_shape=(3,)),
          keras.layers.Softmax(),
      ],
  )
  # saving
  model.save("model.keras")
  # loading
  loaded_model = keras.saving.load_model("model.keras")

  x = keras.random.uniform((10, 3))
  assert np.allclose(model.predict(x), loaded_model.predict(x))
  ```

## TensorFlow

https://www.tensorflow.org/install/pip

limiting GPU memory usage:

https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth


## Install pakages

```shell
conda create -n menv python=3.10

pip install matplotlib numpy pandas opencv-python  keras tensorflow[and-cuda]

```



## Colab

In [ ]:
# Mount Drive in Colab
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/captcha"

## imports

In [ ]:
# Global
import numpy as np
import matplotlib.pyplot as plt

# for Preprocessing
import cv2 as cv

# for Dataset
import gzip
import struct
import pandas as pd

# for Neural Network
import tensorflow as tf
import keras

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from sklearn.model_selection import train_test_split

from preprocessing import *

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

## Captcha Preprocessing

In [ ]:
image = cv.imread("dataset/captchas/00053.gif")

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
F1 = filter1(image)
F2 = filter2(image)

In [ ]:
plt.imshow(F1)
plt.show()

Clustering

In [ ]:
F3, centers = cluster(F1)

In [ ]:
fig,axes = plt.subplots(1,5,figsize=(10,10))
for i,ax in enumerate(axes.flat):
    ax.imshow(F3[i])

In [ ]:
i = np.copy(F1)

for x, y in centers:
    plt.imshow(cv.circle(i, (y,x), 15, (255, 255, 255)))

## Neural Network

[Handwriting recognition](https://en.wikipedia.org/wiki/Handwriting_recognition)

### Sample

- https://www.kaggle.com/code/achintyatripathi/emnist-letter-dataset-97-9-acc-val-acc-91-78

- https://www.kaggle.com/code/prajwalkanade/emnist-hand-writing-recognition-using-ann




### Train

load the dataset

In [ ]:
train_set =  np.loadtxt('dataset/kaggleemnist/emnist-balanced-train.csv', delimiter=",", dtype=np.int64)
train_labels = train_set[:,0]
train_images = train_set[:,1:]

test_set =  np.loadtxt('dataset/kaggleemnist/emnist-balanced-test.csv', delimiter=",", dtype=np.int64)
test_labels = test_set[:,0]
test_images = test_set[:,1:]

check shape of the dataset

In [ ]:
print("Shape of train_set: \t", train_set.shape)
print("Shape of train_images: \t", train_images.shape)
print("Shape of train_labels: \t", train_labels.shape)
print()
print("Shape of test_set: \t", test_set.shape)
print("Shape of test_images: \t", test_images.shape)
print("Shape of test_labels: \t", test_labels.shape)

Creating label dictionary: Running a loop for ASCII equivalent to character conversion

In [ ]:
label_dictionary = {}

for index, label in np.loadtxt("dataset/kaggleemnist/emnist-balanced-mapping.txt", delimiter=" ", dtype=np.int64):
    label_dictionary[index] = chr(label)

label_dictionary

normalise and reshape and transpose images

In [ ]:
train_images_number = train_images.shape[0]
train_images_height = 28
train_images_width = 28

x_train = train_images.reshape(train_images_number, train_images_height, train_images_width, 1) / 255.0

test_images_number = test_images.shape[0]
test_images_height = 28
test_images_width = 28

x_test = test_images.reshape(test_images_number, test_images_height, test_images_width, 1) / 255.0

transform labels

In [ ]:
number_of_classes = len(np.unique(train_labels))

y_train = tf.keras.utils.to_categorical(train_labels, number_of_classes)
y_test  = tf.keras.utils.to_categorical(test_labels, number_of_classes)

plot the dataset

In [ ]:
fig,axes = plt.subplots(1,5,figsize=(20,20))

for i,ax in enumerate(axes.flat):
    ax.imshow(x_train[i].reshape(28,28))
    ax.set_title(label_dictionary[np.argmax(y_train[i])] + " " + str(train_labels[i]))

the model

In [ ]:
# E4 accuracy: 0.9119 - loss: 0.2270 - val_accuracy: 0.8698 - val_loss: 0.4181 - learning_rate: 2.0000e-04 
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,3,input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(number_of_classes,activation='softmax')
])

model.summary()

In [ ]:
#  E6 - accuracy: 0.9375 - loss: 0.1534 - val_accuracy: 0.8870 - val_loss: 0.3737 - learning_rate: 1.0000e-04
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(tf.keras.layers.MaxPool2D(strides=2))
model.add(tf.keras.layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(84, activation='relu'))
model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

check points

In [ ]:
MCP = ModelCheckpoint('checkpoint2.model.keras', verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')
ES = EarlyStopping(monitor='val_accuracy', min_delta=0, verbose=1, restore_best_weights=True, patience=3, mode='max')
RLP = ReduceLROnPlateau(monitor='val_loss', patience=1, factor=0.2, min_lr=0.0001, verbose=1)

train

In [ ]:
history = model.fit(x_train,
                    y_train,
                    verbose=1,
                    epochs=50,
                    batch_size=16,
                    validation_data=(x_test, y_test),
                    callbacks=[MCP,ES,RLP])

plot history

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')

plt.legend(['accuracy', 'loss','val_accuracy','val_loss'], loc='upper center')

plt.grid(axis='y')
plt.show()

In [ ]:
plt.plot(history.history['learning_rate'])
plt.show()

### Prediction

load the model

In [ ]:
mymodel = keras.models.load_model('checkpoint2.model.keras')

In [ ]:
i = cv.copyMakeBorder(cv.resize(charimg[4], (24,24)) ,2,2,2,2,cv.BORDER_CONSTANT).T / 255.0

In [ ]:
prob = mymodel.predict(i.reshape((1,28,28,1)))

label_dictionary[np.argmax(prob)]

In [ ]:
plt.imshow(i)

In [ ]:
captcha = []

for i in range(len(charimg)):
    input = cv.copyMakeBorder(cv.resize(charimg[i], (24,24)) ,2,2,2,2,cv.BORDER_CONSTANT).T / 255.0
    prob = model.predict(input.reshape((1,28,28,1)))
    captcha.append(label_dictionary[np.argmax(prob)])


plt.imshow(img)
plt.title(" ".join(captcha))
plt.show()

In [ ]:
def predict(number):
    
    text = []
    
    for i in range(number):
    
        img = cv.imread(f"dataset/captchas/{i+1:05}.gif")
    
        c,_ = cluster(filter1((img)))
    
        captcha = []
        
        for j in range(len(c)):
            
            inputimg = cv.copyMakeBorder(cv.resize(c[j], (24,24)) ,2,2,2,2,cv.BORDER_CONSTANT).T / 255.0
            
            prob = mymodel.predict(inputimg.reshape((1,28,28,1)), verbose=0)
            
            captcha.append(label_dictionary[np.argmax(prob)])
        
        text.append((i+1,f"{''.join(captcha)}"))
        
        del img, c, captcha

    return text


In [ ]:
predict(1000)

In [ ]:
    fig,axes = plt.subplots(5,5,figsize=(20,20))


In [ ]:
for i in unzip(

In [ ]:
        ax.imshow(img, vmin=0, vmax=255)
        ax.set_title(f"{i+1}-{''.join(captcha)}")
